In [19]:
import pandas as pd

In [1]:
similarity_data = pd.read_csv('./User_Similarity.csv')

In [2]:
similarity_data.head()

,1,2,3,4,5,6,7,8,9,10,...,9992,9993,9994,9995,9996,9997,9998,9999,10000,user_handle
0,1.000000e+00,-2.552584e-17,-4.872686e-17,1.344390e-17,4.190053e-17,1.366378e-16,-8.171220e-18,2.884149e-18,2.165848e-17,2.350236e-17,...,2.112283e-17,2.544800e-17,-4.315699e-17,1.277025e-16,-4.486297e-18,3.523477e-17,-4.341599e-17,-5.013873e-18,-2.538672e-17,1
1,-2.552584e-17,1.000000e+00,-2.337695e-17,5.088368e-17,-3.937430e-17,2.753025e-18,-2.937217e-17,-1.205818e-17,-1.267291e-17,3.243841e-17,...,-1.313792e-18,-2.136013e-17,2.795564e-17,1.204739e-17,1.914894e-18,7.981119e-19,-3.982355e-18,1.692444e-18,-9.597738e-18,2
2,-4.872686e-17,-2.337695e-17,1.000000e+00,-3.183218e-16,4.152206e-17,-2.602085e-17,5.589062e-17,-9.866240e-18,2.634696e-17,-6.246360e-17,...,4.943962e-17,-7.324141e-17,-3.659182e-17,2.856873e-17,-4.076600e-17,6.288373e-18,5.702903e-17,4.336809e-18,1.778092e-17,3
3,1.344390e-17,5.088368e-17,-3.183218e-16,1.000000e+00,2.120251e-17,-1.043545e-17,7.264155e-18,-6.993104e-18,4.006635e-17,-5.475221e-18,...,-2.450297e-17,-1.891848e-17,-2.732189e-17,-4.445229e-18,-1.734723e-17,2.038300e-17,1.994932e-17,-8.239937e-18,-1.843144e-18,4
4,4.190053e-17,-3.937430e-17,4.152206e-17,2.120251e-17,1.000000e+00,1.074994e-17,-5.320826e-18,-1.017406e-17,-3.190783e-17,-1.264813e-16,...,2.180301e-17,1.024273e-16,1.758529e-17,1.249257e-17,-1.727731e-17,-5.749005e-17,5.985816e-18,2.498861e-18,2.145765e-17,5


In [34]:
cols_to_sort = similarity_data.user_handle.tolist()

In [35]:
similarity_data.index = similarity_data.user_handle.values

In [40]:
from tqdm import tqdm
top_k = 100
user_based_top_similarity = []
max_val = 0
for val in tqdm(list(cols_to_sort)):
    user_based_top_similarity.append(similarity_data[str(val)]. \
                                     sort_values(ascending = False ).index.tolist()[1:top_k+1])

100%|██████████| 10000/10000 [00:16<00:00, 597.95it/s]


In [45]:
top_columns = ['top_'+str(ind) for ind in range(1, 101)]

In [48]:
similarity_data.reset_index(inplace = True)

In [49]:
user_based_top_similarity_df = pd.DataFrame(user_based_top_similarity,columns  = top_columns)
user_based_top_similarity_df['user_handle'] = similarity_data['user_handle']

In [50]:
user_based_top_similarity_df.head()

,top_1,top_2,top_3,top_4,top_5,top_6,top_7,top_8,top_9,top_10,...,top_92,top_93,top_94,top_95,top_96,top_97,top_98,top_99,top_100,user_handle
0,3512,753,8589,258,8061,5649,7859,729,5688,1159,...,2597,7575,9921,8970,9556,9650,9759,696,4438,1
1,5129,2220,2135,8892,6988,9206,6670,4227,2765,3881,...,8378,3637,5211,3209,1050,9236,5493,8806,1787,2
2,5175,4174,3162,3826,2997,9853,1017,5558,5218,915,...,2838,4909,3578,4411,7906,3830,119,7505,680,3
3,146,3536,4829,989,227,7865,338,3050,6264,9003,...,696,578,1487,8710,6581,5037,2612,2411,4790,4
4,6153,9085,7681,9675,6756,1839,7752,355,9219,8463,...,9353,8471,2969,2003,8899,3964,5919,1034,4166,5


In [51]:
user_based_top_similarity_df.to_csv('user_top_similarity.csv', index = False)

In [7]:
from configparser import ConfigParser
 
def config(filename='database.ini', section='postgresql'):
    # create a parser
    parser = ConfigParser()
    # read config file
    parser.read(filename)
 
    # get section, default to postgresql
    db = {}
    if parser.has_section(section):
        params = parser.items(section)
        for param in params:
            db[param[0]] = param[1]
    else:
        raise Exception('Section {0} not found in the {1} file'.format(section, filename))
 
    return db

In [11]:
import psycopg2
 
def connect():
    """ Connect to the PostgreSQL database server """
    conn = None
    try:
        # read connection parameters
        params = config()
 
        # connect to the PostgreSQL server
        print('Connecting to the PostgreSQL database...')
        conn = psycopg2.connect(**params)
      
        # create a cursor
        cur = conn.cursor()
        
   # execute a statement
        print('PostgreSQL database version:')
        cur.execute('SELECT version()')
 
        # display the PostgreSQL database server version
        db_version = cur.fetchone()
        print(db_version)
       
       # close the communication with the PostgreSQL
        cur.close()
    except (Exception, psycopg2.DatabaseError) as error:
        print(error)
    finally:
        if conn is not None:
            conn.close()
            print('Database connection closed.')
connect()

Connecting to the PostgreSQL database...
PostgreSQL database version:
('PostgreSQL 10.12 on x86_64-apple-darwin, compiled by i686-apple-darwin11-llvm-gcc-4.2 (GCC) 4.2.1 (Based on Apple Inc. build 5658) (LLVM build 2336.11.00), 64-bit',)
Database connection closed.


In [21]:
user_based_top_similarity_df = pd.read_csv('user_top_similarity.csv')

In [56]:
def create_tables():
    """ create tables in the PostgreSQL database"""
    user_command = "CREATE TABLE users ( user_handle integer PRIMARY KEY , "
    for val in range(1, 101):
        col = 'top_' + str(val)
        user_command = user_command + col + ' integer not null, '
    user_command = user_command[:-2] + ')'
    conn = None
    try:
        # read the connection parameters
        params = config()
        # connect to the PostgreSQL server
        conn = psycopg2.connect(**params)
        cur = conn.cursor()
        # create table 
        cur.execute(user_command)
        # close communication with the PostgreSQL database server
        cur.close()
        # commit the changes
        conn.commit()
    except (Exception, psycopg2.DatabaseError) as error:
        print(error)
    finally:
        if conn is not None:
            conn.close()

In [57]:
create_tables()

relation "users" already exists



In [58]:
from io import StringIO
conn = psycopg2.connect(**config())
sio = StringIO()
sio.write(user_based_top_similarity_df.to_csv(index=None, header=None))  # Write the Pandas DataFrame as a csv to the buffer
sio.seek(0)  # Be sure to reset the position to the start of the stream

# Copy the string buffer to the database, as if it were an actual file
with conn.cursor() as c:
    c.copy_from(sio, "users", columns=user_based_top_similarity_df.columns, sep=',')
    conn.commit()

In [67]:
import json
from psycopg2.extras import RealDictCursor
def get_users(user_id):
    """ query data from the users table """
    conn = None
    try:
        params = config()
        conn = psycopg2.connect(**params)
        cur = conn.cursor(cursor_factory=RealDictCursor)
        cur.execute("SELECT * FROM users where user_handle =" + str(user_id))
        result = json.dumps(cur.fetchall())
        cur.close()
        return result
    except (Exception, psycopg2.DatabaseError) as error:
        print(error)
    finally:
        if conn is not None:
            conn.close()
get_users(2)

'[{"user_handle": 2, "top_1": 5129, "top_2": 2220, "top_3": 2135, "top_4": 8892, "top_5": 6988, "top_6": 9206, "top_7": 6670, "top_8": 4227, "top_9": 2765, "top_10": 3881, "top_11": 1368, "top_12": 9164, "top_13": 2434, "top_14": 4056, "top_15": 4884, "top_16": 6871, "top_17": 697, "top_18": 5549, "top_19": 3742, "top_20": 8943, "top_21": 2287, "top_22": 4734, "top_23": 2504, "top_24": 5092, "top_25": 9735, "top_26": 5995, "top_27": 1367, "top_28": 8149, "top_29": 6152, "top_30": 1572, "top_31": 3874, "top_32": 4709, "top_33": 1114, "top_34": 4691, "top_35": 2851, "top_36": 2496, "top_37": 5517, "top_38": 2286, "top_39": 2611, "top_40": 106, "top_41": 4449, "top_42": 122, "top_43": 9216, "top_44": 4932, "top_45": 2421, "top_46": 71, "top_47": 4200, "top_48": 9081, "top_49": 2332, "top_50": 7105, "top_51": 8182, "top_52": 3824, "top_53": 9651, "top_54": 9833, "top_55": 6057, "top_56": 5102, "top_57": 3640, "top_58": 3677, "top_59": 8372, "top_60": 7480, "top_61": 4530, "top_62": 6574, "